# Load Modules

In [50]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2

# Load Source list files

In [51]:
## This is for CV-old dataset
source = pd.read_csv('../source_list/pulsar_old_updated/0-150.csv').iloc[: , 1:]
source


,type,SRC_ID,A_NAME,B_DEC,B_NAME,B_RA,DEC,RA,num_obs,XOFFSET_
0,PULSAR,PL0001,PSR_J0007+7303_,73.052259,2CXO_J000701.5+730308_,1.756555,73.052056,1.757083,1,0.015
1,PULSAR,PL0002,PSR_J0023+0923_,9.389989,2CXO_J002316.8+092323_,5.820373,9.389963,5.820326,2,0.003
2,PULSAR,PL0003,PSR_J0100-7211_,-72.192701,2CXO_J010043.0-721133_,15.179341,-72.192667,15.179292,20,0.002
3,PULSAR,PL0004,PSR_J0101-6422_,-64.375178,2CXO_J010111.0-642230_,15.296214,-64.375047,15.296318,1,0.008
4,PULSAR,PL0005,PSR_J0146+6145_,61.750834,2CXO_J014622.4+614503_,26.593348,61.750778,26.593417,1,0.004
...,...,...,...,...,...,...,...,...,...,...
133,PULSAR,PL0134,PSR_J2238+5903_,59.061979,2CXO_J223827.9+590343_,339.616483,59.061333,339.617792,1,0.056
134,PULSAR,PL0135,PSR_J2241-5236_,-52.610060,2CXO_J224142.0-523636_,340.425038,-52.610063,340.425077,1,0.001
135,PULSAR,PL0136,PSR_J2256-1024_,-10.409532,2CXO_J225656.3-102434_,344.234879,-10.409551,344.234971,1,0.006
136,PULSAR,PL0137,PSR_J2301+5852_,58.878949,2CXO_J230108.2+585244_,345.284506,58.879014,345.284542,1,0.004


In [52]:
source = source[source['num_obs']>0]
source = source.replace(np.nan , np.nan).sort_values('num_obs' , ascending=False).reset_index(drop=True)
display(source)

,type,SRC_ID,A_NAME,B_DEC,B_NAME,B_RA,DEC,RA,num_obs,XOFFSET_
0,PULSAR,PL0081,PSR_J1745-2900_,-29.007799,2CXO_J174540.0-290028_,266.416843,-29.008304,266.417359,142,0.041
1,PULSAR,PL0099,PSR_J1833-1034,-10.568861,2CXO_J183333.5-103407_,278.389860,-10.568800,278.389900,128,0.004
2,PULSAR,PL0015,PSR_J0537-6910_,-69.172182,2CXO_J053747.4-691019_,84.447521,-69.172189,84.447567,35,0.001
3,PULSAR,PL0020,PSR_J0633+1746_,17.770521,2CXO_J063354.2+174613_,98.475897,17.770253,98.475638,25,0.022
4,PULSAR,PL0026,PSR_J0835-4510_,-45.176291,2CXO_J083520.6-451034_,128.835887,-45.176354,128.835881,22,0.004
...,...,...,...,...,...,...,...,...,...,...
130,PULSAR,PL0059,PSR_J1531-5610_,-56.182037,2CXO_J153127.8-561055_,232.866240,-56.182036,232.866254,1,0.001
131,PULSAR,PL0058,PSR_J1514-4946_,-49.771096,2CXO_J151419.0-494615_,228.579492,-49.770977,228.579642,1,0.009
132,PULSAR,PL0054,PSR_J1429-5911,-59.193342,2CXO_J142958.5-591136_,217.493755,-59.193500,217.493900,1,0.010
133,PULSAR,PL0052,PSR_J1417-4402_,-44.049235,2CXO_J141730.5-440257_,214.377275,-44.049278,214.377500,1,0.010


# Read Chandra data files 

In [53]:
ch_names = []
ch_ra = []
ch_dec = []
offset = []
src_id_list = [] 
n_obs_list = []
df_combined = pd.DataFrame()
for index in tqdm(range(len(source))):
    s = source['A_NAME'][index]
    df = pd.read_csv('../data/all_data/pulsar_old_all/'+s+'.csv' , index_col='index').sort_values('sepn').reset_index(drop=True)
    #display(df)
    top_match = df.loc[0]
    # Select only the source , which has highest cross-matching 
    # And create an array with this seperation value to be stored in original source list table
    # such that source list table have chandra source name , number of actual obs  and seperation.
    # Select all the obs of this cross match and append in all data file

    # Put actual source name , and give source id and obs id in this data
    df_select = df[df['name']==top_match['name']].reset_index(drop=True)
    n_obs_cur = len(df_select)
    obs_id = ['PL_'+str(index)+'_obs_'+str(i) for i in range(len(df_select))]
    src_id = ['PL_'+str(index)]*len(df_select)
    df_select.insert(0 ,'src_id' , src_id)
    df_select.insert(1 ,'obs_id' , obs_id)
    df_select.insert(2, 'num_obs' , [str(len(df_select))]*len(df_select))
    df_select.insert(3 , 'src_n' , [s]*len(df_select))
    top_row = df_select.iloc[0]
    ch_names.append(top_row['name'])
    ch_ra.append(top_row['ra'])
    ch_dec.append(top_row['dec'])
    offset.append(top_row['sepn'])
    n_obs_list.append(len(df_select))
    src_id_list.append('PL_'+str(index))
    #display(df_select)
    df_combined = df_combined.append(df_select)
source_updated = source.copy().reset_index(drop=True)
source_updated.insert(7 , 'chandra_name' , ch_names)
source_updated.insert(8 , 'ch_ra' , ch_ra)
source_updated.insert(9 , 'ch_dec' , ch_dec)
source_updated.insert(10 , 'xoffset' , offset)
source_updated.insert(11 , 'num_obs_used' , n_obs_list)
source_updated = source.reset_index(drop=True)
display(source_updated)
#display(df_combined)
df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.set_index('obs_id')
df_combined

100%|██████████| 135/135 [00:06<00:00, 20.25it/s]


,type,SRC_ID,A_NAME,B_DEC,B_NAME,B_RA,DEC,RA,num_obs,XOFFSET_
0,PULSAR,PL0081,PSR_J1745-2900_,-29.007799,2CXO_J174540.0-290028_,266.416843,-29.008304,266.417359,142,0.041
1,PULSAR,PL0099,PSR_J1833-1034,-10.568861,2CXO_J183333.5-103407_,278.389860,-10.568800,278.389900,128,0.004
2,PULSAR,PL0015,PSR_J0537-6910_,-69.172182,2CXO_J053747.4-691019_,84.447521,-69.172189,84.447567,35,0.001
3,PULSAR,PL0020,PSR_J0633+1746_,17.770521,2CXO_J063354.2+174613_,98.475897,17.770253,98.475638,25,0.022
4,PULSAR,PL0026,PSR_J0835-4510_,-45.176291,2CXO_J083520.6-451034_,128.835887,-45.176354,128.835881,22,0.004
...,...,...,...,...,...,...,...,...,...,...
130,PULSAR,PL0059,PSR_J1531-5610_,-56.182037,2CXO_J153127.8-561055_,232.866240,-56.182036,232.866254,1,0.001
131,PULSAR,PL0058,PSR_J1514-4946_,-49.771096,2CXO_J151419.0-494615_,228.579492,-49.770977,228.579642,1,0.009
132,PULSAR,PL0054,PSR_J1429-5911,-59.193342,2CXO_J142958.5-591136_,217.493755,-59.193500,217.493900,1,0.010
133,PULSAR,PL0052,PSR_J1417-4402_,-44.049235,2CXO_J141730.5-440257_,214.377275,-44.049278,214.377500,1,0.010


,src_id,num_obs,src_n,sepn,instrument,detect_stack_id,name,ra,dec,obsid,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
PL_0_obs_0,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,1561,...,5.07e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_1,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14943,...,5.07e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_2,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14942,...,5.07e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_3,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14941,...,5.07e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_4,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14704,...,5.07e-07,NaN,NaN,NaN,71,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_130_obs_0,PL_130,1,PSR_J1531-5610_,0.000000,ACIS,acisfJ1531267m561009_001,2CXO J153127.8-561055,232.8662,-56.182040,9078,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PL_131_obs_0,PL_131,1,PSR_J1514-4946_,0.000000,ACIS,acisfJ1514075m494548_001,2CXO J151419.0-494615,228.5795,-49.771100,12534,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PL_132_obs_0,PL_132,1,PSR_J1429-5911,0.000000,ACIS,acisfJ1429571m591136_001,2CXO J142958.5-591136,217.4938,-59.193340,14825,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


## Save raw table

In [54]:
df_combined.replace(np.nan , '')
df_combined.to_csv('raw_data/pl_old_raw_data.csv' ,)

## Filter flags

In [55]:
from features import col_to_drop

In [56]:
df = pd.read_csv('raw_data/pl_old_raw_data.csv' , index_col='obs_id')
filter_flags = [
    'pileup_flag',
    'mstr_sat_src_flag',
    'mstr_streak_src_flag',
    'sat_src_flag',
    'streak_src_flag',
    ]
df = df.replace({
    '  NAN' : np.nan , 
    '      NaN' : np.nan , 
    'FALSE' : False , 
    'False' : False , 
    ' TRUE' : True , 
    
    })
print('BEFORE FILTERING')
display(df)
for f in filter_flags:
    df = df[df[f]==False]
print('AFTER FILTERING')
#display(df)
df = df.drop(columns=col_to_drop)
df

BEFORE FILTERING


,src_id,num_obs,src_n,sepn,instrument,detect_stack_id,name,ra,dec,obsid,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
PL_0_obs_0,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,1561,...,5.070000000000001e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_1,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14943,...,5.070000000000001e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_2,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14942,...,5.070000000000001e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_3,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14941,...,5.070000000000001e-07,NaN,NaN,NaN,71,0,0,5,0,0
PL_0_obs_4,PL_0,71,PSR_J1745-2900_,0.003074,ACIS,acisfJ1745401m290028_001,2CXO J174540.0-290028,266.4168,-29.007800,14704,...,5.070000000000001e-07,NaN,NaN,NaN,71,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_130_obs_0,PL_130,1,PSR_J1531-5610_,0.000000,ACIS,acisfJ1531267m561009_001,2CXO J153127.8-561055,232.8662,-56.182040,9078,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PL_131_obs_0,PL_131,1,PSR_J1514-4946_,0.000000,ACIS,acisfJ1514075m494548_001,2CXO J151419.0-494615,228.5795,-49.771100,12534,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PL_132_obs_0,PL_132,1,PSR_J1429-5911,0.000000,ACIS,acisfJ1429571m591136_001,2CXO J142958.5-591136,217.4938,-59.193340,14825,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


AFTER FILTERING


,src_id,num_obs,src_n,sepn,name,ra,dec,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
PL_0_obs_0,PL_0,71,PSR_J1745-2900_,0.003074,2CXO J174540.0-290028,266.4168,-29.007800,1.0140000000000001e-07,0.0,1.979e-07,...,0.0001654,6.0,1.0,1.227e-06,0.0,0.0,5.070000000000001e-07,NaN,NaN,NaN
PL_0_obs_1,PL_0,71,PSR_J1745-2900_,0.003074,2CXO J174540.0-290028,266.4168,-29.007800,0.0,0.0,2.0829999999999998e-07,...,0.0001654,6.0,1.0,1.227e-06,0.0,0.0,5.070000000000001e-07,NaN,NaN,NaN
PL_0_obs_2,PL_0,71,PSR_J1745-2900_,0.003074,2CXO J174540.0-290028,266.4168,-29.007800,2.651e-07,0.0,5.301e-07,...,0.0001654,6.0,1.0,1.227e-06,0.0,0.0,5.070000000000001e-07,NaN,NaN,NaN
PL_0_obs_3,PL_0,71,PSR_J1745-2900_,0.003074,2CXO J174540.0-290028,266.4168,-29.007800,0.0,0.0,3.154e-07,...,0.0001654,6.0,1.0,1.227e-06,0.0,0.0,5.070000000000001e-07,NaN,NaN,NaN
PL_0_obs_4,PL_0,71,PSR_J1745-2900_,0.003074,2CXO J174540.0-290028,266.4168,-29.007800,0.0,0.0,1.8480000000000001e-07,...,0.0001654,6.0,1.0,1.227e-06,0.0,0.0,5.070000000000001e-07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_130_obs_0,PL_130,1,PSR_J1531-5610_,0.000000,2CXO J153127.8-561055,232.8662,-56.182040,0.0,0.0,4.336e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_131_obs_0,PL_131,1,PSR_J1514-4946_,0.000000,2CXO J151419.0-494615,228.5795,-49.771100,1.118e-06,4.934e-07,1.743e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_132_obs_0,PL_132,1,PSR_J1429-5911,0.000000,2CXO J142958.5-591136,217.4938,-59.193340,1.358e-07,0.0,2.7149999999999996e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
from features import arranged_cols  , id_cols 

In [58]:
df_id = df[id_cols]
df_val = df.drop(columns=id_cols)
val_col_list = df_val.columns.to_list()
for v in val_col_list:
    df_val.loc[:,v] = pd.to_numeric(df_val.loc[:,v])
df_val

,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,streak_src_flag,likelihood_b,likelihood_h,likelihood_m,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
PL_0_obs_0,1.014000e-07,0.000000e+00,1.979000e-07,1.304000e-16,0.000000e+00,2.608000e-16,False,1872.97,1898.61,26.04,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_1,0.000000e+00,0.000000e+00,2.083000e-07,0.000000e+00,0.000000e+00,2.553000e-16,False,151.37,197.69,NaN,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_2,2.651000e-07,0.000000e+00,5.301000e-07,1.847000e-15,8.394000e-17,3.609000e-15,False,71.48,70.61,NaN,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_3,0.000000e+00,0.000000e+00,3.154000e-07,0.000000e+00,0.000000e+00,4.295000e-16,False,51.48,46.87,NaN,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_4,0.000000e+00,0.000000e+00,1.848000e-07,2.650000e-16,0.000000e+00,5.415000e-16,False,172.03,256.23,3.27,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_130_obs_0,0.000000e+00,0.000000e+00,4.336000e-07,NaN,NaN,NaN,False,27.90,NaN,25.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_131_obs_0,1.118000e-06,4.934000e-07,1.743000e-06,1.633000e-15,7.205000e-16,2.546000e-15,False,34.04,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_132_obs_0,1.358000e-07,0.000000e+00,2.715000e-07,3.019000e-16,0.000000e+00,6.037000e-16,False,61.19,63.21,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_ok = pd.concat([df_id , df_val] , axis=1)
df_ok

,src_id,num_obs,sepn,name,src_n,ra,dec,livetime,significance,likelihood,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
PL_0_obs_0,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,35796.9,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_1,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,18202.1,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_2,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,19827.7,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_3,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,19819.3,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
PL_0_obs_4,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,36341.0,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,0.0,0.0,5.070000e-07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_130_obs_0,PL_130,1,0.000000,2CXO J153127.8-561055,PSR_J1531-5610_,232.8662,-56.182040,9911.3,3.31,27.89532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_131_obs_0,PL_131,1,0.000000,2CXO J151419.0-494615,PSR_J1514-4946_,228.5795,-49.771100,9909.2,2.63,84.06728,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_132_obs_0,PL_132,1,0.000000,2CXO J142958.5-591136,PSR_J1429-5911,217.4938,-59.193340,24735.6,4.19,63.20605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
from features import phot_flux , phot_flux_hilim , phot_flux_lolim , en_flux , en_flux_hilim , en_flux_lolim
flux_cols =  en_flux + en_flux_hilim + en_flux_lolim 
df_flux_filt = df_ok.copy().replace(0 , np.nan).replace(np.nan , -9987)
for f in flux_cols:
    df_flux_filt = df_flux_filt.query(f+'<1e-12')
df_flux_filt = df_flux_filt.replace(-9987 , np.nan)
df_flux_filt = df_flux_filt.query('likelihood > 50')
df_flux_filt = df_flux_filt.query('significance > 3')
df_flux_filt

,src_id,num_obs,sepn,name,src_n,ra,dec,livetime,significance,likelihood,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
PL_0_obs_0,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,35796.9,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,NaN,NaN,5.070000e-07,NaN,NaN,NaN
PL_0_obs_1,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,18202.1,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,NaN,NaN,5.070000e-07,NaN,NaN,NaN
PL_0_obs_2,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,19827.7,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,NaN,NaN,5.070000e-07,NaN,NaN,NaN
PL_0_obs_3,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,19819.3,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,NaN,NaN,5.070000e-07,NaN,NaN,NaN
PL_0_obs_4,PL_0,71,0.003074,2CXO J174540.0-290028,PSR_J1745-2900_,266.4168,-29.007800,36341.0,107.91,21810.19000,...,0.000165,6.0,1.0,0.000001,NaN,NaN,5.070000e-07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_128_obs_0,PL_128,1,NaN,2CXO J162806.9-320548,PSR_J1628-3205_,247.0291,-32.096930,20027.3,11.99,988.89650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_129_obs_0,PL_129,1,0.003074,2CXO J161436.5-223031,PSR_J1614-2230_,243.6521,-22.508630,19769.6,9.04,515.33310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PL_132_obs_0,PL_132,1,NaN,2CXO J142958.5-591136,PSR_J1429-5911,217.4938,-59.193340,24735.6,4.19,63.20605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_flux_filt.to_csv('filtered_data/pl_old_data.csv')